---
**License**

 TestInterp

 Mon Jan 25 20:56:00 2020\
 Copyright  2021\
 Eduardo ...\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 TestInterp\
 Copyright (C) 2021 Eduardo ...\
 Copyright (C) 2021 Sandro Dias Pinto Vitenti <vitenti@uel.br>


 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

In [1]:
try:
  import gi
  gi.require_version('NumCosmo', '1.0')
  gi.require_version('NumCosmoMath', '1.0')
except:
  pass

import sys
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats


from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
%matplotlib inline

In [2]:
__name__ = "NcContext"

Ncm.cfg_init ()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

In [3]:
dim = 20
p = np.random.random_sample((dim,))
print (p)

[ 0.43552112  0.1216263   0.24682055  0.17714792  0.32958429  0.53740971
  0.96421669  0.46012116  0.46138395  0.70755715  0.28783639  0.67849817
  0.47271837  0.88935956  0.40074473  0.65481791  0.18758079  0.69959009
  0.10242707  0.42278872]


In [4]:
rng = Ncm.RNG.new (None)

fmodel = Ncm.ModelMVND.new (dim)
fdata = Ncm.DataGaussCovMVND.new_full (dim, 0.1, 0.4, 10.0, -1.0, 1.0, rng)

fdata.props.use_norma = True

mset = Ncm.MSet.new_array ([fmodel])
mset.param_set_all_ftype (Ncm.ParamType.FREE)
mset.prepare_fparam_map ()

mset.fparams_set_array (p)
mset.pretty_log ()

#----------------------------------------------------------------------------------
# Model[00000]:
#   - NcmModelMVND : MVND
#----------------------------------------------------------------------------------
# Model parameters
#   -  mu_0[00]:  0.435521123843542   [FREE]
#   -  mu_1[01]:  0.121626296841453   [FREE]
#   -  mu_2[02]:  0.246820549649832   [FREE]
#   -  mu_3[03]:  0.177147918854317   [FREE]
#   -  mu_4[04]:  0.329584291020229   [FREE]
#   -  mu_5[05]:  0.537409709307441   [FREE]
#   -  mu_6[06]:  0.964216693297617   [FREE]
#   -  mu_7[07]:  0.460121155119321   [FREE]
#   -  mu_8[08]:  0.461383952796846   [FREE]
#   -  mu_9[09]:  0.707557150438806   [FREE]
#   - mu_10[10]:  0.287836393049871   [FREE]
#   - mu_11[11]:  0.678498167964171   [FREE]
#   - mu_12[12]:  0.472718373588213   [FREE]
#   - mu_13[13]:  0.889359563649455   [FREE]
#   - mu_14[14]:  0.400744732609671   [FREE]
#   - mu_15[15]:  0.654817914883412   [FREE]
#   - mu_16[16]:  0.187580794965636   [FREE]
#   - 

In [5]:
y_a = []
x_a = []
xn_a = []




interp = Ncm.StatsDistNdKDEStudentt.new (len (p), False, 200)

for a in range (5000):
    v, N = fdata.gen (mset, None, None, rng)
    interp.add_obs (v)
    y_a.append (fdata.m2lnL_val (mset))
    x_a.append (v.dup_array ())

for a in range (5000):
    v, N = fdata.gen (mset, None, None, rng)
    xn_a.append (v.dup_array ())

y_a = np.array (y_a)
x_a = np.array (x_a)
xn_a = np.array (xn_a)

fdata.y.set_array (p)

fdata.m2lnL_val (mset)

-34.47593073502856

In [6]:
#interp.prepare ()
interp.prepare_interp (Ncm.Vector.new_array (y_a))


In [ ]:
def NormalizeData(data,sample):
    return (data - np.min(sample)) / (np.max(sample) - np.min(sample))
        
yi_a = []
ya_a = []
yi_b = []

for x in x_a:
    mset.fparams_set_array (x)
    fdata.prepare (mset)
    yi_a.append (interp.eval_m2lnp (Ncm.Vector.new_array (x)))
    ya_a.append (fdata.m2lnL_val (mset))

yi_a = np.array (yi_a)
ya_a = np.array (ya_a)

diff = yi_a - ya_a
print(diff)


norm, cont = scipy.stats.mode (diff)
plt.figure (figsize=(14, 7))

bla = plt.hist(np.abs(diff), bins = 60)


In [ ]:
yi_a = []
ya_a = []

for x in xn_a:
    mset.fparams_set_array (x)
    fdata.prepare (mset)
    yi_a.append (interp.eval_m2lnp (Ncm.Vector.new_array (x)))
    ya_a.append (fdata.m2lnL_val (mset))

yi_a = np.array (yi_a) 
ya_a = np.array (ya_a)

diff = yi_a -  ya_a
reldiff = np.abs (diff / ya_a)
for a in range(0,10):
    print (reldiff[a])

#print (ya_a)
#print (yi_a)
print(np.max(ya_a), np.max(diff), np.max(yi_a), np.max(reldiff))
print (np.sum ((reldiff < 0.5)/len(reldiff))*100.0)


In [ ]:
plt.figure (figsize=(14, 7))
bla = plt.hist((reldiff), bins = np.linspace (-2,8,60))
